In [4]:
# Dependencies
import pandas as pd
import requests
import json
import time

# Google developer API key
from api_keys import g_key

In [5]:
# lists to append data
restaurant_name = []
place_id = []
price_level = []
restaurant_rating = []
address = []
failed_urls = []

# geocoordinates
locations = ["40.90401600057265, -74.51677868878483", 
              "40.78740421010181, -74.69633648208703", 
              "40.7965015895215, -74.48107312589904",
              "40.86949536737688, -74.41206525573352",
              "40.761144784151384, -74.40760205837368",
              "40.88831523848244, -74.6498162551271"]
target_radius = 8000
target_type = "restaurant"

# go through each of the 6 Morris Countyu locations
for target_coordinates in locations:

    # base url
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_coordinates}&radius={target_radius}&type={target_type}&key={g_key}"

    # run a request using our params dictionary
    response = requests.get(url)

    # print the response url, avoid doing for public github repos in order to avoid exposing key
    #print(response.url)

    # convert response to json
    places_data = response.json()

    # Loop through each page of results
    while True:
        #print(len(places_data["results"]))  
        # loop through all the results in the page
        for item in places_data["results"]:
            # find the restaurant name
            restaurant_name.append(item['name'])
            
            # find the places_id
            place_id.append(item['place_id'])
            try:
                price_level.append(item['price_level'])
            except:
                price_level.append("")
            
            # find the restaurant rating
            try:
                restaurant_rating.append(item['rating'])
            except:
                restaurant_rating.append("")

        # next page
        try:
            next_page_token = places_data['next_page_token']
        except:
            break
        
        # sleep timer needed for next_page_token to work
        time.sleep(6)
        
        # url with next page token to find next page results
        url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_coordinates}&radius={target_radius}&type={target_type}&key={g_key}&pagetoken={next_page_token}"

        # run a request using our params dictionary
        response = requests.get(url)
        #print(response.url)
        
        places_data = response.json()
        #print(json.dumps(places_data, indent=4, sort_keys=True))
        # save failed urls to find later
        if places_data["status"] == "INVALID_REQUEST":
            failed_urls.append(response.url)


20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20


In [6]:
print(failed_urls)


[]


In [7]:
# Dataframe to store all retrieved restaurant data
google_data_df = pd.DataFrame({'restaurant_name':restaurant_name,
                              'place_id':place_id,
                              'price_level':price_level,
                              'restaurant_rating':restaurant_rating})
google_data_df

,restaurant_name,place_id,price_level,restaurant_rating
0,Popeyes Louisiana Kitchen,ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1,3.7
1,Buffalo Wild Wings,ChIJAYlPufkKw4kRr7-IhJCnC3Y,2,4.2
2,Auntie Anne's,ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1,4.2
3,Tabor Road Tavern,ChIJg8StYWinw4kRMFNslu1L9xg,3,4.4
4,Subway,ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1,3.4
...,...,...,...,...
355,Anthony & Sons Bakery & Italian Deli,ChIJ1yZeabefw4kRfvmIeEYa2xY,2,4.5
356,Pizza Express Mt. Arlington,ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1,3.7
357,Shell,ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2,3.1
358,Dunkin',ChIJNcJSc_2fw4kRIA4rycWRc58,1,4.2


In [12]:
# use Place Details API to find the restaurants google maps url, website and address
restaurant_url = []
address = []
google_maps_url = []

for index, row in google_data_df.iterrows():
    
    # base url
    url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={row['place_id']}&fields=formatted_address,url,website&key={g_key}"

    # run a request using our params dictionary
    response = requests.get(url)

    # convert response to json
    places_data = response.json()
    google_maps_url.append(places_data["result"]['url'])
    
    #find the restaurant website
    try:
        restaurant_url.append(places_data["result"]['website'])
    except:
        restaurant_url.append("")
    
    # find the restaurant address if it exists
    try:
        address.append(places_data["result"]['formatted_address'])
    except:
        address.append("")

google_data_df['google_maps_url'] = google_maps_url
google_data_df['restaurant_website'] = restaurant_url
google_data_df['address'] = address
google_data_df

,Unnamed: 0,Unnamed: 0.1,restaurant_name,place_id,price_level,restaurant_rating,google_maps_url,address,restaurant_website
0,0,0,Popeyes Louisiana Kitchen,ChIJ3dh_VfcKw4kR3lrLsxKB7tY,1.0,3.7,https://maps.google.com/?cid=15487458085899557598,Rockaway Townsquare Mall-301 Mt Hope Road - Su...,https://www.popeyes.com/store-locator/store/re...
1,1,1,Buffalo Wild Wings,ChIJAYlPufkKw4kRr7-IhJCnC3Y,2.0,4.2,https://maps.google.com/?cid=8506076560359997359,"343 Mt Hope Ave, Rockaway, NJ 07866, USA",https://www.buffalowildwings.com/en/locations/...
2,2,2,Auntie Anne's,ChIJ3fcDtfsKw4kRsy2xOBfI0aQ,1.0,4.2,https://maps.google.com/?cid=11876493694413647283,"301 Mt Hope Ave Suite 1048, Rockaway, NJ 07866...",https://www.auntieannes.com/nj/rockaway/119-1?...
3,3,3,Tabor Road Tavern,ChIJg8StYWinw4kRMFNslu1L9xg,3.0,4.4,https://maps.google.com/?cid=1798990059960816432,"510 Tabor Rd, Morris Plains, NJ 07950, USA",http://www.taborroadtavern.com/
4,4,4,Subway,ChIJ6YlEVO8Jw4kR4n_aDP4NbIc,1.0,3.4,https://maps.google.com/?cid=9758189877393522658,"559 E Main St Store # 2, Denville, NJ 07834, USA",https://order.subway.com/en-us/restaurant/2197...
...,...,...,...,...,...,...,...,...,...
355,355,355,Anthony & Sons Bakery & Italian Deli,ChIJ1yZeabefw4kRfvmIeEYa2xY,2.0,4.5,https://maps.google.com/?cid=1646938978706323838,"15 NJ-10, Succasunna, NJ 07876, USA",http://anthonyandsonsbakerynj.com/
356,356,356,Pizza Express Mt. Arlington,ChIJM4tBOgJ1w4kRa9U1mz6xgfI,1.0,3.7,https://maps.google.com/?cid=17474442911624320363,"312 Howard Blvd, Mt Arlington, NJ 07856, USA",https://pizzaexpressmtarlington.com/
357,357,357,Shell,ChIJWf-qdf2fw4kR_MdkDkTL6Jo,2.0,3.1,https://maps.google.com/?cid=11162395169597474812,"274 US-46, Mine Hill Township, NJ 07803, USA",https://find.shell.com/us/fuel/12395193-274-rt...
358,358,358,Dunkin',ChIJNcJSc_2fw4kRIA4rycWRc58,1.0,4.2,https://maps.google.com/?cid=11489687353021894176,"274 Rte 46, Mine Hill Township, NJ 07803, USA",https://smart.link/abu8tr12sc1y6?utm_source=go...


In [13]:
# save to csv file
google_data_df.to_csv('../Resources/google_data.csv')